In [59]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import LogisticRegression
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score

# read data

In [60]:
Df = pd.read_csv('BTC-USD.csv')
Df

Date          Open          High           Low         Close  \
0     2014-09-17    465.864014    468.174011    452.421997    457.334015   
1     2014-09-18    456.859985    456.859985    413.104004    424.440002   
2     2014-09-19    424.102997    427.834991    384.532013    394.795990   
3     2014-09-20    394.673004    423.295990    389.882996    408.903992   
4     2014-09-21    408.084991    412.425995    393.181000    398.821014   
...          ...           ...           ...           ...           ...   
2708  2022-02-15  42586.464844  44667.218750  42491.035156  44575.203125   
2709  2022-02-16  44578.277344  44578.277344  43456.691406  43961.859375   
2710  2022-02-17  43937.070313  44132.972656  40249.371094  40538.011719   
2711  2022-02-18  40552.132813  40929.152344  39637.617188  40030.976563   
2712  2022-02-19  40022.132813  40246.027344  40010.867188  40126.429688   

         Adj Close       Volume  
0       457.334015     21056800  
1       424.440002     34483200  
2       394.795990     37919700  
3       408.903992     36863600  
4       398.821014     26580100  
...            ...          ...  
2708  44575.203125  22721659051  
2709  43961.859375  19792547657  
2710  40538.011719  26246662813  
2711  40030.976563  23310007704  
2712  40126.429688  22263900160  

[2713 rows x 7 columns]

# close price, drop missing values

In [61]:
Df=Df[['Close']]
Df= Df.dropna() 
Df.Close.plot(figsize=(10,5))
plt.ylabel("Prices")
plt.show()

# data preprocessing

## calculate the 3 days average stock price, 9 days average and 15 days average stock price and combine as X values

In [62]:
Df['S_3'] = Df['Close'].shift(1).rolling(window=3).mean() 
Df['S_9']= Df['Close'].shift(1).rolling(window=9).mean() 
Df['S_15']= Df['Close'].shift(1).rolling(window=15).mean() 

Df= Df.dropna() 
X = Df[['S_3','S_9','S_15']] 
X.head()

S_3         S_9        S_15
15  382.008667  399.746884  405.611265
16  381.876994  393.000329  400.127130
17  372.732991  385.923330  395.798596
18  354.483327  376.733551  391.403263
19  336.296000  367.409664  385.510331

# close price as y values

In [63]:
y = Df['Close']
y.head()

15    375.071991
16    359.511993
17    328.865997
18    320.510010
19    330.079010
Name: Close, dtype: float64

# split to training and testing data

In [64]:
t=0.8
t = int(t*len(Df)) 

X_train = X[:t] 
y_train = y[:t]  

X_test = X[t:]
y_test = y[t:]

In [70]:
param_grid = {
    'fit_intercept': [True, False],
    'normalize': [True, False],
    'copy_X': [True, False],
    'n_jobs': [None, -1],
}

grid_search = GridSearchCV(estimator=LinearRegression(), param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

/Users/lekshyuen/Documents/anaconda/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
/Users/lekshyuen/Documents/anaconda/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous 

/Users/lekshyuen/Documents/anaconda/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
/Users/lekshyuen/Documents/anaconda/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous 

# train the model

In [71]:
linear = LinearRegression(**best_params).fit(X_train,y_train)

/Users/lekshyuen/Documents/anaconda/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


# prediction


In [72]:
predicted_price = linear.predict(X_test)

In [73]:
df_predicted_price = pd.DataFrame(predicted_price,index=y_test.index,columns = ['predicted_price'])  
df_predicted_price


predicted_price
2173     11412.820072
2174     11428.790495
2175     11582.581714
2176     11638.439951
2177     11818.494147
...               ...
2708     42166.635320
2709     43054.530524
2710     43760.694156
2711     43014.430936
2712     41279.227484

[540 rows x 1 columns]

# evaluate the model

In [74]:
accuracy = linear.score(X_test, y_test)*100
accuracy 

98.47516868080672

In [75]:
error = mean_squared_error(predicted_price, y_test)
error

3814260.3536154265

# visualize

In [76]:
df_predicted_price.plot(figsize=(15,10), color='blue', marker='o', linestyle='dashed')
y_test.plot(color='red')

plt.legend(['predicted_price','actual_price'])  
plt.xlabel("Time")  
plt.ylabel("Price") 
plt.show()